In [9]:
from Bio import SeqIO
import numpy as np
import pandas as pd
import primer3

# get reverse complementary sequence for reverse primer
def getRevComp(sequence):
    dicComp = {'A':'T',
                'T':'A',
                'C':'G',
                'G':'C',
                'N':'N'}
    revComp = []
    for i in sequence[::-1]:
        revComp.append(dicComp[i])
    return ''.join(revComp)

In [10]:
parse = [x for x in SeqIO.parse('./matK_prank.fasta.best.fas','fasta')]
# parse = [x for x in SeqIO.parse('./matK_prank.best.nex','nexus')]

def ExtractSeqBetweenSNP(parse):
    seq      = []
    for i in parse:
        seq.append([nt for nt in i.seq])
    nex_seq  = np.array(seq).T
    df       = pd.DataFrame(nex_seq, columns=[x.id for x in parse])
    
    # make SNP index list
    listSNPix = []
    for ix in df.index:
        if len(set(df.loc[ix].values)) != 1:
            listSNPix.append(ix)
    listSNPix.append(len(df))
    
    # make text file of seq between SNPs
    with open('./seq_between_snps.txt','w') as f:
        print('start_loc','length','sequence',file=f)
        try:
            for ix in range(len(listSNPix)):
                piece_len = listSNPix[ix+1]-listSNPix[ix]-1
                if piece_len >= 18:
                    print(listSNPix[ix]+1,piece_len,
                          parse[0].seq[listSNPix[ix]+1:listSNPix[ix]+piece_len+1],
                          file=f)
        except IndexError:
            pass

In [11]:
ExtractSeqBetweenSNP(parse)

In [12]:
def ExtractCandidatePrimer(SeqBetweenSNPs):
    df_seq_between_snps = pd.read_csv(SeqBetweenSNPs,sep=' ')
    dic_candidate_primer = {
        'sequence'         : [],
        'reverse_sequence' : [],
        'start_loc'        : [],
        'length'           : [],
        'Tm'               : [],
        'GCcontents (%)'   : [],
        'hairpin'          : []
    }
    # extract candidate primers : > 18 length
    for ix in range(len(df_seq_between_snps)):
        i = 0
        while i < 50:
            if len(df_seq_between_snps.loc[ix]['sequence'][0+i:20+i]) in range(18,23):

                sequence         = df_seq_between_snps.loc[ix]['sequence'][0+i:20+i]
                reverse_sequence = getRevComp(sequence)
                start_loc        = int(df_seq_between_snps.loc[ix]['start_loc'])+i
                length           = len(sequence)
                Tm               = primer3.calcTm(sequence)
                GCcontents       = (sequence.count('C')+sequence.count('G'))/len(sequence)*100
                hairpin          = primer3.calcHairpin(sequence)

                dic_candidate_primer['sequence'].append(sequence)
                dic_candidate_primer['reverse_sequence'].append(reverse_sequence)
                dic_candidate_primer['start_loc'].append(start_loc)
                dic_candidate_primer['length'].append(length)
                dic_candidate_primer['Tm'].append(Tm)
                dic_candidate_primer['GCcontents (%)'].append(GCcontents)
                dic_candidate_primer['hairpin'].append(hairpin)
                i = i+1
            else:
                break
    df_candidate_primer = pd.DataFrame(dic_candidate_primer)
    
    # setting primer conditions
    m1  = df_candidate_primer['Tm'] >= 40            # normally 50~60 °C
    m2  = df_candidate_primer['length'] >= 18        # normally 18~24 mer
    m3  = df_candidate_primer['GCcontents (%)'] > 40 # normally 50 %
    m   = m1 & m2 & m3
    df_candidate_primer_m = df_candidate_primer[m]
    return df_candidate_primer_m

In [15]:
ExtractCandidatePrimer('./seq_between_snps.txt')

,sequence,reverse_sequence,start_loc,length,Tm,GCcontents (%),hairpin
23,GCTCATTACTTGTCAAACGG,CCGTTTGACAAGTAATGAGC,172,20,49.420918,45.000000,"ThermoResult(structure_found=False, tm=0.00, d..."
57,TGCTGGATACATGATGTTCC,GGAACATCATGTATCCAGCA,510,20,49.413737,45.000000,"ThermoResult(structure_found=True, tm=49.41, d..."
58,GCTGGATACATGATGTTCCC,GGGAACATCATGTATCCAGC,511,20,50.208945,50.000000,"ThermoResult(structure_found=True, tm=49.41, d..."
59,CTGGATACATGATGTTCCCA,TGGGAACATCATGTATCCAG,512,20,48.666130,45.000000,"ThermoResult(structure_found=True, tm=49.41, d..."
60,TGGATACATGATGTTCCCAC,GTGGGAACATCATGTATCCA,513,20,48.985822,45.000000,"ThermoResult(structure_found=True, tm=42.65, d..."
61,GGATACATGATGTTCCCACT,AGTGGGAACATCATGTATCC,514,20,48.658713,45.000000,"ThermoResult(structure_found=False, tm=0.00, d..."
122,ATTGGGTCATTGGCCAGAGC,GCTCTGGCCAATGACCCAAT,1128,20,54.778507,55.000000,"ThermoResult(structure_found=True, tm=38.76, d..."
123,TTGGGTCATTGGCCAGAGCT,AGCTCTGGCCAATGACCCAA,1129,20,55.785520,55.000000,"ThermoResult(structure_found=True, tm=37.51, d..."
124,TGGGTCATTGGCCAGAGCTA,TAGCTCTGGCCAATGACCCA,1130,20,54.910782,55.000000,"ThermoResult(structure_found=True, tm=37.51, d..."
125,GGGTCATTGGCCAGAGCTAA,TTAGCTCTGGCCAATGACCC,1131,20,53.843751,55.000000,"ThermoResult(structure_found=True, tm=40.64, d..."


In [16]:
df_candidate_primer_m = ExtractCandidatePrimer('./seq_between_snps.txt')

primer_f = df_candidate_primer_m.loc[61]['sequence']
primer_r = df_candidate_primer_m.loc[149]['reverse_sequence']

In [18]:
primer_f,primer_r

('GGATACATGATGTTCCCACT', 'CAAACTGGCTTGCTAATAGG')

In [17]:
primer3.calcHeterodimer(primer_f,primer_r)

ThermoResult(structure_found=True, tm=-46.05, dg=-1452.28, dh=-26700.00, ds=-81.40)